    from IPython import get_ipython; ip = get_ipython()
    %reload_ext pidgin.tangle

`import pidgin.tangle` modifies the `get_ipython().input_transformer_manager` to accept __Markdown__ source.  The `pidgin.tangle` module
exports:
    
* `pidgin.tangle.markdown_to_python` - is a semi-lossless __Markdown__ to __Python__ converter.


In [1]:
    import ast, mistune, textwrap, functools, itertools, IPython, importnb
    __all__ = 'markdown_to_python',

`quote` wrotes non code objects in triple ticks.

In [2]:
    def quote(str, punc=''):
        str, leading_ws = ''.join(str), []
        lines = str.splitlines(True)
        _ = '"""'
        if _ in str: _ = "'''"
        if not str.strip(): _ = punc = ''
        while lines and (not lines[0].strip()): leading_ws.append(lines.pop(0))    
        str = ''.join(lines)
        end = len(str.rstrip())
        str, ending_ws = str[:end], str[end:]
        if str and str.endswith(_[0]): str += ' '                    
        return F"{''.join(leading_ws)}{_}{str}{_}{punc}{ending_ws}"

`get_first_line` get the first non-`iter`able strings in `lines`

In [3]:
    def get_first_line(lines, line=''):
        for line in lines or ['']: 
            if line.strip(): break
        return line

`get_line_indent` computes the indent of a string.

In [4]:
    def get_line_indent(line):  return len(line) - len(line.lstrip())

The `Markdown` transformer must combine code and non-code.  It must handle logic to properly indent tangled code and non-code.

In [5]:
    class Markdown(mistune.Markdown):
        def render(self, text):
            """Initialize the """
            if text.lstrip().startswith('%%'):
                return text
            
            self.renderer.original = ''.join(text).splitlines()
            self.renderer.final, self.renderer.buffer, self.renderer.min_indent = [], [], 4
            self.parse(text)
            while self.renderer.original: self.renderer.buffer.append(self.renderer.original.pop(0))
            self.renderer.block_code('', None, punc=';')
            return '\n'.join(self.renderer.final)

The __Lexer__ s only consider coarse features of the markdown spec.  

In [6]:
    class BlockLexer(mistune.BlockLexer): 
        list_rules = default_rules = [
            'newline', 'fences', 'block_code', 'block_quote', 'list_block', 'block_html', 'paragraph', 'text']
        def parse_fences(self, m):
            """This version of parse_fences makes it possible to identify code fences."""
            super().parse_fences(m)
            self.tokens[-1]['lang'] = self.tokens[-1]['lang'] or ''
    class InlineLexer(mistune.InlineLexer):  
        inline_html_rules = default_rules = ['text']

In [7]:
    def _has_return(code):
        code = '\n'.join(code)
        if 'return ' not in code: return False
        code = importnb.loader.dedent(code)
        try:
            node = ast.parse(code)
            while hasattr(node, 'body'): node = node.body[-1]
            return isinstance(node, ast.Return)
        except: ...  

In [8]:
    class Renderer(mistune.Renderer):
        def text(self, text):  return text
        
        def newline(self):
            self.buffer += ['']
            return ''
        
        def paragraph(self, text):
            line, lines = '', text.splitlines()
            while lines and not lines[-1].strip():  lines.pop()
            while lines:
                line = lines.pop(0)
                while self.original and line not in self.original[0]:
                    self.buffer += [self.original.pop(0)]
            if self.original and line in self.original[0]:  self.buffer += [self.original.pop(0)]
            return ''

        def block_code(self, text, lang, punc=''):
            if isinstance(lang, str): 
                return self.paragraph(text)
            
            code, lines = [], text.splitlines()
            
            # Extract the first line of the current code block.
            first_line = get_first_line(text.splitlines())
            
            # Populate the buffer until the 
            while self.original and (
                (lang or first_line) not in self.original[0]
                or not self.original[0].strip()
            ): self.buffer += [self.original.pop(0)]
                
            # Remove empty lines
            while lines and not lines[0].strip(): lines.pop(0)
            
            # Drop the lines from the original body
            if self.original:
                while lines:
                    line = lines.pop(0)
                    while line not in self.original[0]: 
                        code += [self.original.pop(0)]
                    code += [self.original.pop(0)]
                    
            # Construct the code we'll 
            code = '\n'.join(code + ['']).lstrip('```')
                            
            # The body goes about the code, the buffer is non-code.
            body = '\n'.join(self.buffer + [''])
            
            # The previous last line append
            last_line = get_first_line(reversed(self.final))
            
            # The current indent level so far.
            prior_indent = get_line_indent(last_line)

            # Does the last line enter a block statement
            definition = last_line.rstrip().endswith(':')
            returns = _has_return(self.final)

            this_indent = get_line_indent(get_first_line(code.splitlines()))
            
            if this_indent < self.min_indent:
                code = textwrap.indent(code, ' '*(self.min_indent-this_indent))
                this_indent = get_line_indent(get_first_line(code.splitlines()))

            # Assign the minimum indent 
            if body.strip() and not self.min_indent: 
                self.min_indent = this_indent
                
            # Normalize the indent we'll assign the body+code
            indent = max(self.min_indent, (returns and min or max)(prior_indent, this_indent))            
            
            # Wrap non-code in block quotes if it was not explicitly assigned
            if not (last_line.endswith('"""') or last_line.endswith("'''")): 
                body = quote(body, punc)
            
            # Indent the body
            body = textwrap.indent(body, ' '*indent)
            
            # A hanging indent if the there was a defintion
            if definition and prior_indent == indent: 
                body = ' '*4+body
                
            # Cell Magics
            if code.lstrip().startswith('%%'):
                # Cell magics can be split across __Markdown__ blocks.  With this 
                # approach conditional blocks can be used with magics.
                code = textwrap.indent(importnb.loader.dedent(code), ' '*this_indent)

            self.final.extend(body.splitlines() + code.splitlines())
            self.buffer = []
            return ''            

## Exports

`markdown_to_python` converts __Markdown__ to __Python__ in a semi lossless way.

In [9]:
    def markdown_to_python(str)->"Valid Python Source": 
        renderer = Renderer(escape=False)
        if str.lstrip().startswith('%%'): return str
        return Markdown(renderer=renderer, 
            inline=InlineLexer, 
            block=BlockLexer).render(str)

In [10]:
    def transformer(lines: "that end with a newline."): 
        # Always add a new line
        return markdown_to_python(''.join(lines + ['\n'])).splitlines(1)

In [11]:
    def load_ipython_extension(ip=None):
        ip = ip or IPython.get_ipython()
        ip.input_transformer_manager.cleanup_transforms = [transformer] + [
            object for object in ip.input_transformer_manager.cleanup_transforms
            if object not in {transformer, IPython.core.inputtransformer2.classic_prompt}
        ]

    def unload_ipython_extension(ip=None):
        ip = ip or IPython.get_ipython()
        ip.input_transformer_manager.cleanup_transforms = [
            object for object in ip.input_transformer_manager.cleanup_transforms
            if object is not transformer]
    if __name__ == '__main__': load_ipython_extension()